In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [ ]:
def get_links(sdt, edt): # YYYYMMDDHHMMSS
    url = 'https://api.gdeltproject.org/api/v2/doc/doc?query="COP26"&mode=artlist&maxrecords=250&startdatetime='+sdt+'&enddatetime='+edt+'&trans=googtrans' 
    response = requests.get(url)
    driver.get(url)
    page = response.text
    page = driver.page_source
    return page

In [ ]:
month = '202111' #YYYYMM
start_day = 1 #DD
end_day =2 #DD


news=pd.DataFrame(columns=['url', 'arttitle', 'source','date', 'time', 'language', 'country'])

for day in range(start_day,end_day+1,1):

    for hour in range(0,24,1):

        links=pd.DataFrame(columns=['url', 'arttitle', 'source','date', 'time', 'language', 'country'])

        start = str.zfill(str(hour),2)
        days = str.zfill(str(day),2)
        sdt = month+days+start+'0000'
        edt = month+days+start+'5959'

        page = get_links(sdt, edt)

        soup = BeautifulSoup(page, "html5lib")

        for index, link in enumerate(soup.find_all('a')):
            urls = link.get("href")
            links.loc[index,'url'] = urls

        for index, span in enumerate(soup.find_all("span", class_="arttitle")):
            arttitles = span.text
            links.loc[index,'arttitle'] = arttitles

        for index, span in enumerate(soup.find_all("span", class_="sourceinfo")):
            sourceinfos = span.text.split()
            links.loc[index,'source'] = sourceinfos[0]
            links.loc[index,'date'] = sourceinfos[4]
            links.loc[index,'time'] = sourceinfos[5]
            links.loc[index,'language'] = sourceinfos[6]
            try:
                try:
                    links.loc[index,'country'] = sourceinfos[7] + " " + sourceinfos[8]
                except:
                    links.loc[index,'country'] = sourceinfos[7]
            except:
                pass
            
        news = news.append(links, ignore_index = True)


In [ ]:
news.to_csv('month.csv')

In [ ]:
news_hl = pd.read_csv('news_headline_EN.csv', index_col = 0)
news_hl

In [ ]:
import re # Regular expression library
import string

In [ ]:
# Replace punctuation with a white space and lower case

alphanumeric = lambda x: re.sub(r'[^A-Za-z ]+', '', x)
lowcase = lambda x: x.lower()

In [ ]:
news_hl['clean'] = news_hl['arttitle']

In [ ]:
news_hl['clean'] = news_hl.clean.map(alphanumeric)

In [ ]:
news_hl.clean

In [ ]:
news_hl['clean'] = news_hl.clean.map(lowcase)

In [ ]:
news_hl.clean

In [ ]:
label = [e[:75]+"..." for e in news_hl['clean']]

In [ ]:
from textblob import TextBlob

In [ ]:
news_hl['sentiment'] = np.nan
news_hl['polarity'] = np.nan

In [ ]:
for index, hl in enumerate(news_hl['clean']):

    news_hl.loc[index,'sentiment'] = TextBlob(hl).sentiment.subjectivity
    news_hl.loc[index,'polarity'] = TextBlob(hl).sentiment.polarity

In [ ]:
news_hl.to_csv('news_hl_clean.csv')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [ ]:
# Incorporate stop words when creating the count vectorizer

vec = TfidfVectorizer(stop_words='english', min_df=10)
# vec = CountVectorizer(stop_words='english', min_df=10)

X = vec.fit_transform(news_hl.clean)

In [ ]:
news_tokens = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

In [ ]:
news_words = news_tokens.columns.tolist()

In [ ]:
news_tokens

In [ ]:
# from nltk.stem import WordNetLemmatizer
# import nltk
# nltk.download('wordnet')

In [ ]:
# wnl = WordNetLemmatizer()

In [ ]:
# for w in news_tokens.columns:
#     news_tokens = news_tokens.rename({w: wnl.lemmatize(w)}, axis=1)

# takes too long to run - limited benefit

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
nmf_model = NMF(20)
news_topics = nmf_model.fit_transform(news_tokens)

In [ ]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20'],
             columns = vec.get_feature_names())
topic_word

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):           
        print('{}: '.format(ix) + ','.join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
display_topics(nmf_model, vec.get_feature_names(), 10)

In [ ]:
for ix in range(0,20,1):
    top_news_topics = pd.DataFrame(news_topics.round(5),
             index = label,
             columns = ['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20'])
    print("\nTopic ", ix)
    print(top_news_topics.iloc[:, ix].sort_values(ascending=False).head(20))

In [ ]:
map_topics = pd.DataFrame(news_topics.round(5),
             columns = ['Fight for Climate Change', \
                        'Net Zero', \
                        'World Leadership', \
                        'Deforestation Pledge', \
                        'Climate Summit Deficiency', \
                        'United Kingdom', \
                        'Climate Action', \
                        'Global Warming', \
                        'Clean Energy', \
                        'Activists', \
                        'Carbon Emissions', \
                        'China', \
                        'Green Living', \
                        'Commitment', \
                        'Greenhouse Gas', \
                        'Climate Conference Kick-off', \
                        'None', \
                        'Fossil Fuel', \
                        'Climate Crisis', \
                        'Greta Thunberg'])


In [ ]:
news_hl_sent_topic = pd.concat([news_hl, map_topics], axis=1)

In [ ]:
news_hl_sent_topic.to_csv('news_hl_sent_topic.csv')

In [ ]:
from corextopic import corextopic as ct
from corextopic import vis_topic as vt

In [ ]:
ct_model = ct.Corex(n_hidden=20, words=news_words,vmax_iter=200, verbose=False, seed=1)

ct_model.fit(X, words=news_words, docs=news_hl.clean, anchors=[['green', 'living'],['pledge', 'deforestation'], \
                         ['carbon','emissions'], ['human','swan'], ['sea','level'], \
                         ['global','warming'], ['climate', 'change', 'fight'],['climate', 'crisis'], \
                         ['net', 'zero'], 'china', 'action',['united','kingdom'],['blah', 'activist'], \
                         ['clean','energy'],['greenhouse','gas'],['fossil','fuel']], anchor_strength=2)

In [ ]:
# Print all topics from the CorEx topic model
ct_topics = ct_model.get_topics()
for n,topic in enumerate(ct_topics):
    ct_topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(ct_topic_words))

In [ ]:
for ix in range(0,20,1):
    top_news_topics = pd.DataFrame(ct_model.get_top_docs(topic=ix, n_docs=20, sort_by='log_prob'))
    

    print("\nTopic ", ix)
    print(top_news_topics)